# Visualization Example

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
DIR = "/workspace/nflbigdatabowl2023"
sys.path.append(DIR)

In [3]:
import math
from ast import literal_eval

import pandas as pd
import matplotlib.pyplot as plt

from src.pipeline.flows.main import main_flow
from src.visualization.interactive_play_selector import create_interactive_play_selector

In [4]:
_ = main_flow(max_weeks=3, max_games=3, max_plays=3)

22:27:00.979 | INFO    | prefect.engine - Created flow run 'capable-viper' for flow 'main-flow'

22:27:01.199 | INFO    | Flow run 'capable-viper' - Created task run 'read_csv-3609c996-0' for task 'read_csv'

22:27:01.202 | INFO    | Flow run 'capable-viper' - Executing 'read_csv-3609c996-0' immediately...

22:27:01.444 | INFO    | Task run 'read_csv-3609c996-0' - Finished in state Completed()

22:27:01.471 | INFO    | Flow run 'capable-viper' - Created task run 'read_csv-3609c996-1' for task 'read_csv'

...

22:27:12.188 | INFO    | Task run 'write_csv-386fe2af-2' - Finished in state Completed()

22:27:12.216 | INFO    | Flow run 'capable-viper' - Created task run 'write_csv-386fe2af-3' for task 'write_csv'

22:27:12.219 | INFO    | Flow run 'capable-viper' - Executing 'write_csv-386fe2af-3' immediately...

22:27:12.295 | INFO    | Task run 'write_csv-386fe2af-3' - Finished in state Completed()

22:27:12.344 | INFO    | Flow run 'capable-viper' - Finished in state Completed('All states completed.')

In [6]:
df_plays_all = pd.read_csv(f"{DIR}/data/raw/plays.csv")
df_tracking_display_all = pd.read_csv(f"{DIR}/data/outputs/tracking_display.csv")
df_areas_all = pd.read_csv(f"{DIR}/data/outputs/pocket_areas.csv")
df_areas_all["pocket"] = df_areas_all["pocket"].apply(literal_eval)

/workspace/nflbigdatabowl2023/.venv/lib/python3.8/site-packages/prefect/flows.py:206: UserWarning: A flow named 'main-flow' and defined at '/workspace/nflbigdatabowl2023/src/pipeline/flows/main.py:31' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


In [7]:
create_interactive_play_selector(
    df_plays_all,
    df_tracking_display_all,
    df_areas_all,
    continuous_update=True
)

interactive(children=(Dropdown(description='Week', options=(1, 2, 3), value=1), Dropdown(description='Game ID'…